In [1]:
# @title **Demo FullControl Design - Multi-axis tube with backlash compensation and an option to print a spine on the tube** { display-mode: "form"}

# @markdown #### **🡸 Click ▶︎ button to connect to python** --- *connection may take ~20 seconds*
# @markdown  #### **🡸 Click it again** to regenerate the design after changing parameters **(or press *`shift + enter`)*** --- *gcode generation may take ~20 seconds*

# import python packages (if not already imported)
import sys
if 'fullcontrol' not in sys.modules:
  !pip install --no-deps git+https://github.com/FullControlXYZ/fullcontrol --quiet
  # --no-deps is included due to a pip 'dependency resolver' error in colab that began in Feb 2024
  import fullcontrol as fc
  import lab.fullcontrol as fclab
  import lab.fullcontrol.fouraxis as fc4
from google.colab import files
from math import sin, cos, tau, atan, exp, pi, degrees

# @markdown

# @markdown Rotation of the b axis will cause the nozzle to move in the x and z directions, and the amount that it moves depends on how far the tip of the nozzle is away from the axis of rotation

# @markdown Therefore it is important to set this distances here to allow fullcontrol to determine the correct x z values to send to the printer

# @markdown These offsets are for the axis of rotation (B) relative to the nozzle tip when B=0

Rotation_axis_offset_from_nozzle_tip_x = 40.2 # @param {type:"number"} # default 40.2 mm
Rotation_axis_offset_from_nozzle_tip_z = 82 # @param {type:"number"} # default 82.0 mm
Nozzle_size = 0.4 #  @param {type:"slider", min:0.4, max:0.8, step:0.2}
Output = 'Plot' # @param ["Plot", "GCode"]
TubeOrSpine = 'Tube' # @param ["Tube", "Spine"]


# @markdown  This design does not currently adjust extrusion rate to compensate for the layer height being smaller on the inside of bends but that strategy is implemented in other FullControl designs and will be implemented if there is a demand
# @markdown
# @markdown  If your motor gearbox combo has backlash, the design can compensate for it if you write the value here
# @markdown
# @markdown  use the checkboxes to say
# @markdown  - if the nozzle moves in the positive B direction in the last move be starting the print
# @markdown  - whether backlash is compensated when the nozzle changes direction to positive or negative B


b_offset_z = Rotation_axis_offset_from_nozzle_tip_z
b_offset_x = Rotation_axis_offset_from_nozzle_tip_x
design_name = 'FullControl_4axis_demo'

EH = Nozzle_size / 2 if Output == 'GCode' else Nozzle_size*3
EW = Nozzle_size * 1.25

bez_points = [
    fc.Point(x=0, y=0, z=0),
    fc.Point(x=0, y=0, z=30),
    fc.Point(x=-20, y=0, z=60),
    fc.Point(x=80, y=0, z=90),
    fc.Point(x=0, y=0, z=140),
    fc.Point(x=-30, y=0, z=150),]

layers = int(fc.path_length(fclab.bezier(bez_points, 100))/EH)  # use 100 points to calculate bezier path length
centres = fclab.bezier(bez_points, layers)
centres = fc.segmented_path(centres, layers) # make sure medial axis points are perfectly equidistant

radii = fc.linspace(20, 12, layers)
seg_z_angles = [fclab.angleZ(point1, point2) if point2.x>point1.x else -fclab.angleZ(point1, point2) for point1, point2 in zip(centres[:-1], centres[1:])]
angles = seg_z_angles + [seg_z_angles[-1]] # last point has now segment after it, so use the angle of the previous segment

centre_x, centre_y = 120, 100
centres = fc.move(centres, fc.Vector(x=centre_x, y=centre_y))

# vary_thickness = False

steps = []
for layer in range(layers):
    circle = fc.circleXY(centres[layer], radii[layer], tau/4, 6) # define path for each layer
    circle = fclab.rotate(circle, centres[layer], 'y', angles[layer])  # rotate to be normal to the medial axis of the tube for each layer
    circle = fc4.xyz_add_b(circle) # convert points from 3-axis (XYZ) to 4-axis (XYZB)
    for i in range(len(circle)): circle[i].b = degrees(angles[layer]) # Add nozzle-angle data for each point
    # if vary_thickness:
    #   steps.append((fc.ExtrusionGeometry(width=EW*(1+0.25*sin((layer/layers)*tau*5)))))
    steps.extend(circle) # add current layer to the design

###
### SPINE CODE
###
spine_EW, spine_EH = 1, 0.5
nozzle_offset = EW/2 + spine_EH*0.8 # 0.8 allows some squish
if TubeOrSpine == 'Spine':
  spine=[]
  for i in range(len(steps)):
    if i % 7 == 0:
      spine.append(fc.midpoint(steps[i-2], steps[i-3]))
  spine = spine[20:]
  angles = [fclab.angleZ(spine[i], spine[i+1]) if spine[i].x <= spine[i+1].x else -fclab.angleZ(spine[i], spine[i+1]) for i in range(len(spine)-1)]
  angles = [angle +tau/4 for angle in angles]
  angles.append(angles[-1]) # add final angle to be same as previous - couldn't be calcualted based on the next point since there is no next point
  spine_offset = []
  for i in range(len(spine)):
    spine_offset.append(fclab.spherical_to_point(spine[i], nozzle_offset, 0, angles[i])) # offset noozle in direction of current B for each point by half the tube wall width plus the layer height of the spine
    spine_offset[-1] = fc4.xyz_add_b(spine_offset[-1]) # convert point from 3-axis (XYZ) to 4-axis (XYZB)
    spine_offset[-1].b = degrees(angles[i])    # set B for each point
  steps = spine_offset
  steps = fc.flatten([[step, fc.PlotAnnotation(label=f'{step.b:.0f}')] for step in steps])
  # steps = spine + spine_offset
  steps.insert(0, fc.ExtrusionGeometry(width=spine_EW, height=spine_EH))


import re
import math
Backlash_angle_degrees = 1.35 # @param {type:"number"}
Initially_moving_positive = False # @param {type:"boolean"}
Compensate_positive_direction = True # @param {type:"boolean"}
def replace_b_numbers_in_string(input_string: str, backlash_degrees: float, compensate_positive_direction: bool, initially_moving_positive: bool):
    pattern = re.compile(r'(G[01].*?B)(-?\d+(?:\.\d+)?)(?=\s|$)')
    b_offset = backlash_degrees if compensate_positive_direction == initially_moving_positive else 0
    last_b_value = -1e12*initially_moving_positive  # Initialize with negative infinity if first value is supposed to be positive, otherwise set to infinity
    def replace_func(match):
        nonlocal b_offset, last_b_value
        current_b_value = float(match.group(2))
        if math.isclose(current_b_value, last_b_value):
            pass  # keep the current b_offset
        elif compensate_positive_direction:
          b_offset = backlash_degrees if current_b_value > last_b_value else 0
        else:
            b_offset = 0 if current_b_value > last_b_value else backlash_degrees
        last_b_value = current_b_value
        # print(b_offset)
        # print(current_b_value)
        return f"{match.group(1)}{current_b_value + b_offset}"
    # Process each line separately and join them back with '\n'
    updated_lines = [pattern.sub(replace_func, line) for line in input_string.split('\n')]
    return '\n'.join(updated_lines)

if Output == 'Plot':
  steps.append(fc4.PlotAnnotation(point=centres[-1], label='Not all layers shown in this preview'))
  fc4.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.6, color_type='print_sequence'))
else:
  gcode = fc4.transform(steps, 'gcode', fc4.GcodeControls(b_offset_z=b_offset_z, b_offset_x=b_offset_x , initialization_data={
                      'print_speed': 200, 'extrusion_width': EW, 'extrusion_height': EH}))
  gcode = replace_b_numbers_in_string(input_string=gcode, backlash_degrees=Backlash_angle_degrees, compensate_positive_direction=Compensate_positive_direction, initially_moving_positive=Initially_moving_positive)
  open(f'{design_name}.gcode', 'w').write(gcode)
  files.download(f'{design_name}.gcode')


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
